In [1]:
import pandas as pd

import matplotlib.pyplot as plt

from itertools import combinations 

import numpy as np

from category_encoders import MEstimateEncoder

from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score

from xgboost import XGBRegressor

from sklearn.model_selection import RepeatedKFold

from sklearn.feature_selection import mutual_info_regression

from catboost import CatBoostRegressor

from sklearn.preprocessing import LabelEncoder

from scipy.special import inv_boxcox
from scipy.stats import boxcox



In [2]:
train = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Shipping_Cost/train.csv')
test = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Shipping_Cost/test.csv')

t = test['Customer Id']

train['Cost'] = train['Cost'].abs()

test['Cost'] = -1

data1 = pd.concat([train,test],axis=0)



data1 = data1.drop(['Customer Id'],axis=1)

data1.head()

,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",283.29
1,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",159.96
2,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",154.29
3,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",161.16
4,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",159.23


In [3]:
train = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Shipping_Cost/train.csv')
test = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Shipping_Cost/test.csv')


In [4]:
# train['Cost'] = train['Cost'].abs()
# train['Cost'] = np.log(train['Cost'])

In [5]:
# train['Cost'].skew()

In [6]:
# sns.distplot(train['Cost'])

In [7]:
# train['Cost'] = train['Cost'].abs()
# train['Cost'],l = boxcox(train['Cost'])

In [8]:
# train['Cost'].skew()

In [9]:
# import seaborn as sns
# sns.distplot(train['Cost'])

In [10]:
# train['Cost'].describe()

In [11]:
# train[['a','b']].describe()

In [12]:
# x = train['Cost'].values
# pred,l = boxcox(train['Cost'],lmbda = 1)




In [13]:
d = combinations(['International','Express Shipment'],2)




for i, j in d:
     data1[i + '_' + j] = data1[i].astype(str) + '_' + data1[j].astype(str)

In [14]:
d = combinations(['International','Express Shipment','Transport','Installation Included','Fragile'],2)

for i, j in d:
     data1[i + '_' + j] = data1[i].astype(str) + '_' + data1[j].astype(str)

# d = combinations(['International','Express Shipment','Transport','Installation Included','Fragile'],4)
# for i, j,k,l in d:
#      data1[i + '_' + j + '_' + k + '_' + l] = data1[i].astype(str) + '_' + data1[j].astype(str)+ '_'+ data1[k].astype(str) + '_' + data1[l].astype(str)
        
        
d = combinations(['International','Express Shipment','Transport','Installation Included','Fragile'],3)
for i, j,k in d:
     data1[i + '_' + j + '_' + k ] = data1[i].astype(str) + '_' + data1[j].astype(str)+ '_'+ data1[k].astype(str)

In [15]:
d = combinations(['International','Express Shipment','Transport','Installation Included','Fragile','Remote Location'],6)

for i, j,k,l,m,n in d:
     data1['Mix'] = data1[i].astype(str) + '_' + data1[j].astype(str) + '_' + data1[k].astype(str) + '_' + data1[l].astype(str) + '_' + data1[m].astype(str) + '_' + data1[n].astype(str)


In [16]:
data1.fillna({'Artist Reputation':np.mean(data1['Artist Reputation'])\
              ,'Height':np.mean(data1['Height']),'Width':np.mean(data1['Width'])\
              ,'Weight':np.mean(data1['Weight'])}, inplace=True)


dic = {"No_No":'Roadways',"No_Yes":'Airways',"Yes_No":'Waterways',"Yes_Yes":'Airways'}

data1.Transport = data1.Transport.fillna(data1['International_Express Shipment'].map(dic))

a = data1['Remote Location'].value_counts(normalize=True)

data1['Remote Location'] = data1['Remote Location'].fillna(pd.Series(np.random.choice(a.index, 
                                                       p=a.values, size=len(data1))))


In [17]:
from sklearn.impute import KNNImputer
data1['Material'] = data1['Material'].astype(str)
data1['Fragile'] = data1['Fragile'].astype(str)

df = data1[['Height','Weight','Width','Material','Price Of Sculpture','Fragile']]
ord_enc = LabelEncoder()
df['Material'] = ord_enc.fit_transform(df[['Material']])
df['Fragile'] = ord_enc.fit_transform(df[['Fragile']])


knnimp = KNNImputer(n_neighbors = 3)
df = knnimp.fit_transform(df)
data1['Material'] = df[:,3]

C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\DELL\Anaconda3\envs\PythonML\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [18]:
data1['Area'] = data1['Height'] * data1['Width']


data1['Area'] = data1['Area'].astype(int)

# data1['Area1'] = data1['Base Shipping Price'] * data1['Base Shipping Price']

# data1['Area1'] = data1['Area1'].round(3)

data1['Priceperrep'] =  data1['Price Of Sculpture'] / data1['Base Shipping Price'] 

data1['Priceperrep'] = data1['Priceperrep'].round(3)

data1['repperprice'] =  data1['Artist Reputation'] / data1['Price Of Sculpture'] 

data1['repperprice'] = data1['repperprice'].round(3)

data1['PriceperW'] =  data1['Base Shipping Price'] / data1['Weight'] 

data1['PriceperW'] = data1['PriceperW'].round(3)

data1['Price Of Sculpture'] = data1['Price Of Sculpture'] * data1['Price Of Sculpture']

data1['Finalprice'] = 0.5 * data1['Price Of Sculpture'] + 0.5 * data1['Base Shipping Price'] * data1['Base Shipping Price']

data1['Dimension'] = 2 * data1['Height'] * data1['Width'] * 1.5 * data1['Weight']


data1 = data1.drop(['Artist Name'],axis=1)

In [19]:
data1['Scheduled Date'] = pd.to_datetime(data1['Scheduled Date'],format = '%m/%d/%y')

data1['Delivery Date'] = pd.to_datetime(data1['Delivery Date'],format='%m/%d/%y')

data1['time_diff_days'] = (data1['Scheduled Date'] - data1['Delivery Date']).dt.days

data1['time_diff_months'] = (data1['Scheduled Date'].dt.month - data1['Delivery Date'].dt.month)


data1['time_diff_month'] = data1['Scheduled Date'].dt.month 

data1['time_diff_months'] =  data1['Delivery Date'].dt.month

data1.drop(['Scheduled Date','Delivery Date'],axis=1,inplace=True)

In [20]:
data1['Ratio'] =  data1['PriceperW'] / data1['Priceperrep']

data1['Ratio'] = data1['Ratio'].round(2)

for i in ['Priceperrep','PriceperW','Ratio']:
    data1[i] = data1[i].astype(int)
    

In [21]:
num = data1.select_dtypes(['int64','float64']).columns

num = list(num)

num.remove('Cost')
      


In [22]:
# Specify which column to normalize
col_to_normalize = ['Price Of Sculpture','Finalprice','Dimension']

# Log normalization
for i in col_to_normalize:
    # Add log normalized column
    data1[i] = np.log(data1[i])
    
    data1[i] = data1[i].round(1)
    # Drop the original column



In [23]:
def state(cell):
    try :
        cell = cell.split(sep=', ')[1].split()[0]
    
    except :
        cell = cell.split(sep=' ')[1]
    
    return cell

data1['Customer Location']=data1['Customer Location'].apply(state)
    

In [ ]:
cat = data1.select_dtypes('object').columns

cat = list(cat)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

for i in cat:
    data1[i] = ord_enc.fit_transform(data1[[i]])

In [ ]:
test = data1[data1['Cost']== -1]

train = data1[data1['Cost']!= -1]

test_l = test.drop(['Cost'],axis=1)



In [ ]:


# Specify which column to normalize
col_to_normalize = ['Cost']

# Log normalization
for i in col_to_normalize:
    # Add log normalized column
    train[i] = np.log(train[i])
    train[i] = train[i].round(1)
    # Drop the original column



In [ ]:
y = train['Cost']
x = train.drop(['Cost'],axis=1)

In [ ]:
encoder = MEstimateEncoder(return_df = True,m = 2)
encoder.fit(x,y)
x = encoder.transform(x)
test_l = encoder.transform(test_l)

In [ ]:
def make_mi_scores(x, y):
    mi_scores = mutual_info_regression(x, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(x, y)
imp_columns = list(mi_scores.index[:])
x = x[imp_columns]
test_l = test_l[imp_columns]
print(mi_scores)

In [ ]:
from sklearn import model_selection

x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,test_size = 0.20, shuffle = True,random_state = 22)




In [ ]:
def score(y_true, y_pred):
    y_pred = [(np.exp(x)) for x in [i for i in y_pred]]
    y_true = [(np.exp(x)) for x in [i for i in y_true]]
    score = 100*max(0, 1-metrics.mean_squared_log_error(y_true, y_pred))
    return score

In [ ]:
params = {'n_estimators': 800, 'max_depth': 5, 'learning_rate': 0.16}
# regressor = CatBoostRegressor(verbose=0,**params)


# params3 = {'learning_rate':0.5, 'max_depth':3, 'min_child_weight':15, 'n_estimators':100, 'n_jobs':1, 'objective':"reg:squarederror", 'subsample':0.9000000000000001, 'verbosity':0}
# xgb = XGBRegressor(**params3)

# rf = RandomForestRegressor(random_state=22,n_jobs = -1,n_estimators = 800,max_depth = 10,min_samples_leaf = 3) 
  

# #regressor = CatBoostRegressor(verbose=0,**params)

# cv = cross_val_score(regressor, x,y, scoring = make_scorer(score,greater_is_better = True),cv = 5)

# cv
# print(np.mean(cv))

In [ ]:
# print(cv)

In [ ]:
#Individual catboost
regressor = CatBoostRegressor(**params) 
  
#fit the regressor with x and y data 
regressor.fit(x, y)

y_pred_log = regressor.predict(test_l)

y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

test['Customer Id']=t 
test['Cost'] = y_pred

test[['Customer Id','Cost']].to_csv('retry1.csv',index=False)

In [ ]:
for i in range(len(x[0])):
    print(train.columns[i],' ',regressor.feature_importances_[i])

In [ ]:
# #individual xgb
# params3 = {'learning_rate':0.5, 'max_depth':3, 'min_child_weight':15, 'n_estimators':100, 'n_jobs':1, 'objective':"reg:squarederror", 'subsample':0.9000000000000001, 'verbosity':0}
# xgb = XGBRegressor(**params3)

# fit the regressor with x and y data 
# xgb.fit(x, y)

# # y_pred_log = xgb.predict(test_l)

# # y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

# # test['Customer Id']=t 
# # test['Cost'] = y_pred

# # test[['Customer Id','Cost']].to_csv('sub11.csv',index=False)

In [ ]:
#individual random forest
# from sklearn.ensemble import RandomForestRegressor 
  
#  # create regressor object 
# rf = RandomForestRegressor(random_state=22) 
  
# # fit the regressor with x and y data 
# rf.fit(x,y)

# y_pred_log = rf.predict(test_l)

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

# test['Customer Id']=t 
# test['Cost'] = y_pred

# test[['Customer Id','Cost']].to_csv('sub11.csv',index=False)

In [ ]:

# score = 100*max(0, 1-metrics.mean_squared_log_error(y_test, y_pred))
# score


In [ ]:
# # params = rs.best_params_
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import LinearRegression
# params = {'n_estimators': 800, 'max_depth': 4, 'learning_rate': 0.16}
# params1 = {'n_estimators': 800, 'max_depth': 5, 'learning_rate': 0.16}
# params2 = {'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.16}
# #params3 = {'learning_rate':0.5, 'max_depth':3, 'min_child_weight':15, 'n_estimators':100, 'n_jobs':-1, 'objective':"reg:squarederror", 'subsample':0.9000000000000001, 'verbosity':0}
# #xgb = XGBRegressor(**params3)

# regressor = CatBoostRegressor(verbose=0,**params)
# regressor1 = CatBoostRegressor(verbose=0,**params1)
# regressor2 = CatBoostRegressor(verbose=0,**params2)
# #rf = RandomForestRegressor(random_state=22) 

# est = [('c5',regressor),('c6',regressor1),('c7',regressor2)]
# model = StackingRegressor(est,final_estimator = LinearRegression(),n_jobs = -1)

# model.fit(x, y)

# y_pred_log = model.predict(test_l)

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

# test['Customer Id']=t 
# test['Cost'] = y_pred

# test[['Customer Id','Cost']].to_csv('sub20.csv',index=False)

In [ ]:
# b = pd.read_csv('final5.csv')

# a = pd.read_csv('final4.csv')

# new_cost = 0.5  * a['Cost'] + 0.5 * b['Cost']

# c = pd.DataFrame({'Customer Id': a['Customer Id'],'Cost':new_cost})

# c.to_csv('final6.csv',index=False)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform as sp_randFloat
# from scipy.stats import randint as sp_randInt
# from catboost import CatBoostRegressor

 

# cb = CatBoostRegressor(verbose=0)

 

# parameters = {'max_depth':np.arange(4,20,1), 'learning_rate':np.arange(0.01,0.2,0.01), 'n_estimators':np.arange(100,800,100)}

 

# rs = RandomizedSearchCV(estimator = cb, param_distributions = parameters, scoring=make_scorer(score, greater_is_better = True), cv = 7, n_iter = 10, n_jobs = -1, verbose = 10)

 

# rs.fit(x, y)

 

# print("\n The best estimator across ALL searched params:\n",rs.best_estimator_)
# print("\n The best score across ALL searched params:\n",rs.best_score_)
# print("\n The best parameters across ALL searched params:\n",rs.best_params_)